## utillities

`read_input_for_day` function hust reads input for specific day in inputs directory.

The `Solution` class is made to minimize things to do before writing an actual solution.

## Usage
Use `Solution` as the base class for your solutions. 

Call `run` on your class.
E.G 
```python
class Test(Solution):
    day = 1
    ...

Test.run()
```

That thing will basically read the input file from "./inputs/day1.txt", call the `task1` and `task2` methods and print the results.

The `process_input` function used to transform input lines in something more convinient for the particular solution.

In [1]:
! python -m pip install pydantic
from typing import Optional, Any, List
from abc import abstractmethod

BASE_DIR = "/home/jovyan/work/AOC2020"

def read_input_for_day(day: Optional[int]) -> Any:
    if day is None:
        raise Exception("Day is undefined")
    with open(f"{BASE_DIR}/inputs/day{day}.txt") as f:
        return f.readlines()

class Solution:
    day: int
    
    def process_input(self, inputs: List[str]) -> List[Any]:
        return inputs
    
    def __init__(self):
        self.inputs = self.process_input(read_input_for_day(self.day))
    
    @classmethod
    def run(cls):
        obj = cls()
        task1_value = obj.task1()
        print(f"Task 1: {task1_value}")
        task2_value = obj.task2()
        print(f"Task 2: {task2_value}")
    
    @abstractmethod
    def task1(self):
        pass
    
    @abstractmethod
    def task2(self):
        pass

# Day 1 solutions

## Part 1

After saving Christmas five years in a row, you've decided to take a vacation at a nice resort on a tropical island. Surely, Christmas will go on without you.

The tropical island has its own currency and is entirely cash-only. The gold coins used there have a little picture of a starfish; the locals just call them stars. None of the currency exchanges seem to have heard of them, but somehow, you'll need to find fifty of these coins by the time you arrive so you can pay the deposit on your room.

To save your vacation, you need to get all fifty stars by December 25th.

Collect stars by solving puzzles. Two puzzles will be made available on each day in the Advent calendar; the second puzzle is unlocked when you complete the first. Each puzzle grants one star. Good luck!

Before you leave, the Elves in accounting just need you to fix your expense report (your puzzle input); apparently, something isn't quite adding up.

Specifically, they need you to find the two entries that sum to 2020 and then multiply those two numbers together.

For example, suppose your expense report contained the following:

```
1721
979
366
299
675
1456
```
In this list, the two entries that sum to 2020 are 1721 and 299. Multiplying them together produces 1721 * 299 = 514579, so the correct answer is 514579.

Of course, your expense report is much larger. Find the two entries that sum to 2020; what do you get if you multiply them together?

## Part 2

The Elves in accounting are thankful for your help; one of them even offers you a starfish coin they had left over from a past vacation. They offer you a second one if you can find three numbers in your expense report that meet the same criteria.

Using the above example again, the three entries that sum to 2020 are 979, 366, and 675. Multiplying them together produces the answer, 241861950.

In your expense report, what is the product of the three entries that sum to 2020?

In [2]:
class Day1Solution(Solution):
    day = 1
    
    def process_input(self, inputs) -> List[int]:
        return list(map(int, inputs))
    
    def task1(self):
        for i in self.inputs:
            for j in self.inputs:
                if i + j == 2020:
                    return i * j
    
    def task2(self):
        for i in self.inputs:
            for j in self.inputs:
                for k in self.inputs:
                    if i + j + k == 2020:
                        return i * j * k
    
Day1Solution.run()

Task 1: 73371
Task 2: 127642310


# Day 2 solutions


## Part 1
Your flight departs in a few days from the coastal airport; the easiest way down to the coast from here is via toboggan.

The shopkeeper at the North Pole Toboggan Rental Shop is having a bad day. "Something's wrong with our computers; we can't log in!" You ask if you can take a look.

Their password database seems to be a little corrupted: some of the passwords wouldn't have been allowed by the Official Toboggan Corporate Policy that was in effect when they were chosen.

To try to debug the problem, they have created a list (your puzzle input) of passwords (according to the corrupted database) and the corporate policy when that password was set.

For example, suppose you have the following list:
```
1-3 a: abcde
1-3 b: cdefg
2-9 c: ccccccccc
```

Each line gives the password policy and then the password. The password policy indicates the lowest and highest number of times a given letter must appear for the password to be valid. For example, 1-3 a means that the password must contain a at least 1 time and at most 3 times.

In the above example, 2 passwords are valid. The middle password, cdefg, is not; it contains no instances of b, but needs at least 1. The first and third passwords are valid: they contain one a or nine c, both within the limits of their respective policies.

How many passwords are valid according to their policies?

# Part 2

While it appears you validated the passwords correctly, they don't seem to be what the Official Toboggan Corporate Authentication System is expecting.

The shopkeeper suddenly realizes that he just accidentally explained the password policy rules from his old job at the sled rental place down the street! The Official Toboggan Corporate Policy actually works a little differently.

Each policy actually describes two positions in the password, where 1 means the first character, 2 means the second character, and so on. (Be careful; Toboggan Corporate Policies have no concept of "index zero"!) Exactly one of these positions must contain the given letter. Other occurrences of the letter are irrelevant for the purposes of policy enforcement.

Given the same example list from above:

 - 1-3 a: abcde is valid: position 1 contains a and position 3 does not.
 - 1-3 b: cdefg is invalid: neither position 1 nor position 3 contains b.
 - 2-9 c: ccccccccc is invalid: both position 2 and position 9 contain c.

How many passwords are valid according to the new interpretation of the policies?


In [3]:
from collections import namedtuple
from pydantic import BaseModel


class PasswordModel(BaseModel):
    min: int
    max: int
    symbol: str
    password: str


class Day2Solution(Solution):
    day = 2
    
    def process_input(self, inputs) -> List[PasswordModel]:
        new_inputs = []
        for input_str in inputs:
            min_max, symbol_def, password = input_str.split()
            minimum, maximum = min_max.split("-")
            symbol = symbol_def.rstrip(":")
            new_inputs.append(PasswordModel(
                min=int(minimum),
                max=int(maximum),
                symbol=symbol,
                password=password
            ))
        return new_inputs
    
    def task1(self):
        correct = 0
        for i in self.inputs:
            sym_len = len(list(filter(lambda x: x == i.symbol, i.password)))
            if i.min <= sym_len <= i.max:
                correct += 1
        return correct
    
    def task2(self):
        correct = 0
        for i in self.inputs:
            if ((i.password[i.min - 1] == i.symbol or i.password[i.max - 1] == i.symbol) and
                i.password[i.min -1 ] != i.password[i.max - 1]):
                correct += 1
                
        return correct
    
Day2Solution.run()

Task 1: 467
Task 2: 441


# Day 3 solutions

## Part 1

With the toboggan login problems resolved, you set off toward the airport. While travel by toboggan might be easy, it's certainly not safe: there's very minimal steering and the area is covered in trees. You'll need to see which angles will take you near the fewest trees.

Due to the local geology, trees in this area only grow on exact integer coordinates in a grid. You make a map (your puzzle input) of the open squares (.) and trees (#) you can see. For example:
```
..##.......
#...#...#..
.#....#..#.
..#.#...#.#
.#...##..#.
..#.##.....
.#.#.#....#
.#........#
#.##...#...
#...##....#
.#..#...#.#
```
These aren't the only trees, though; due to something you read about once involving arboreal genetics and biome stability, the same pattern repeats to the right many times:
```
..##.........##.........##.........##.........##.........##.......  --->
#...#...#..#...#...#..#...#...#..#...#...#..#...#...#..#...#...#..
.#....#..#..#....#..#..#....#..#..#....#..#..#....#..#..#....#..#.
..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#
.#...##..#..#...##..#..#...##..#..#...##..#..#...##..#..#...##..#.
..#.##.......#.##.......#.##.......#.##.......#.##.......#.##.....  --->
.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#
.#........#.#........#.#........#.#........#.#........#.#........#
#.##...#...#.##...#...#.##...#...#.##...#...#.##...#...#.##...#...
#...##....##...##....##...##....##...##....##...##....##...##....#
.#..#...#.#.#..#...#.#.#..#...#.#.#..#...#.#.#..#...#.#.#..#...#.#  --->
```
You start on the open square (.) in the top-left corner and need to reach the bottom (below the bottom-most row on your map).

The toboggan can only follow a few specific slopes (you opted for a cheaper model that prefers rational numbers); start by counting all the trees you would encounter for the slope right 3, down 1:

From your starting position at the top-left, check the position that is right 3 and down 1. Then, check the position that is right 3 and down 1 from there, and so on until you go past the bottom of the map.

The locations you'd check in the above example are marked here with O where there was an open square and X where there was a tree:
```
..##.........##.........##.........##.........##.........##.......  --->
#..O#...#..#...#...#..#...#...#..#...#...#..#...#...#..#...#...#..
.#....X..#..#....#..#..#....#..#..#....#..#..#....#..#..#....#..#.
..#.#...#O#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#
.#...##..#..X...##..#..#...##..#..#...##..#..#...##..#..#...##..#.
..#.##.......#.X#.......#.##.......#.##.......#.##.......#.##.....  --->
.#.#.#....#.#.#.#.O..#.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#
.#........#.#........X.#........#.#........#.#........#.#........#
#.##...#...#.##...#...#.X#...#...#.##...#...#.##...#...#.##...#...
#...##....##...##....##...#X....##...##....##...##....##...##....#
.#..#...#.#.#..#...#.#.#..#...X.#.#..#...#.#.#..#...#.#.#..#...#.#  --->
```
In this example, traversing the map using this slope would cause you to encounter 7 trees.

Starting at the top-left corner of your map and following a slope of right 3 and down 1, how many trees would you encounter?

## Part 2

Time to check the rest of the slopes - you need to minimize the probability of a sudden arboreal stop, after all.

Determine the number of trees you would encounter if, for each of the following slopes, you start at the top-left corner and traverse the map all the way to the bottom:

    Right 1, down 1.
    Right 3, down 1. (This is the slope you already checked.)
    Right 5, down 1.
    Right 7, down 1.
    Right 1, down 2.

In the above example, these slopes would find 2, 7, 3, 4, and 2 tree(s) respectively; multiplied together, these produce the answer 336.

What do you get if you multiply together the number of trees encountered on each of the listed slopes?

In [4]:
class Day3Solution(Solution):
    day = 3
    
    def process_input(self, inputs):
        return list(map(lambda x: list(x.rstrip("\n")), inputs))
    
    def task1(self, slope_right=3, slope_down=1):
        x, y = 0,0
        height = len(self.inputs)
        trees = 0
        while y < height:
            test_x = x + slope_right
            test_y = y + slope_down
            if test_y >= height:
                break
            row = self.inputs[test_y]
            if row[test_x % len(row)] == "#":
                trees += 1
            x, y = test_x, test_y
        return trees
    
    def task2(self):
        slopes = [
            (1, 1),
            (3, 1),
            (5, 1),
            (7, 1),
            (1, 2)
        ]
        result = 1
        for slope in slopes:
            result *= self.task1(*slope)
        
        return result
Day3Solution.run()

Task 1: 189
Task 2: 1718180100


# Day 4 solutions

## Part 1
You arrive at the airport only to realize that you grabbed your North Pole Credentials instead of your passport. While these documents are extremely similar, North Pole Credentials aren't issued by a country and therefore aren't actually valid documentation for travel in most of the world.

It seems like you're not the only one having problems, though; a very long line has formed for the automatic passport scanners, and the delay could upset your travel itinerary.

Due to some questionable network security, you realize you might be able to solve both of these problems at the same time.

The automatic passport scanners are slow because they're having trouble detecting which passports have all required fields. The expected fields are as follows:

    byr (Birth Year)
    iyr (Issue Year)
    eyr (Expiration Year)
    hgt (Height)
    hcl (Hair Color)
    ecl (Eye Color)
    pid (Passport ID)
    cid (Country ID)

Passport data is validated in batch files (your puzzle input). Each passport is represented as a sequence of key:value pairs separated by spaces or newlines. Passports are separated by blank lines.

Here is an example batch file containing four passports:

```
ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in
```

The first passport is valid - all eight fields are present. The second passport is invalid - it is missing hgt (the Height field).

The third passport is interesting; the only missing field is cid, so it looks like data from North Pole Credentials, not a passport at all! Surely, nobody would mind if you made the system temporarily ignore missing cid fields. Treat this "passport" as valid.

The fourth passport is missing two fields, cid and byr. Missing cid is fine, but missing any other field is not, so this passport is invalid.

According to the above rules, your improved system would report 2 valid passports.

Count the number of valid passports - those that have all required fields. Treat cid as optional. In your batch file, how many passports are valid?

## Part 2

The line is moving more quickly now, but you overhear airport security talking about how passports with invalid data are getting through. Better add some data validation, quick!

You can continue to ignore the cid field, but each other field has strict rules about what values are valid for automatic validation:

    byr (Birth Year) - four digits; at least 1920 and at most 2002.
    iyr (Issue Year) - four digits; at least 2010 and at most 2020.
    eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
    hgt (Height) - a number followed by either cm or in:
        If cm, the number must be at least 150 and at most 193.
        If in, the number must be at least 59 and at most 76.
    hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
    ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
    pid (Passport ID) - a nine-digit number, including leading zeroes.
    cid (Country ID) - ignored, missing or not.

Your job is to count the passports where all required fields are both present and valid according to the above rules. Here are some example values:
```
byr valid:   2002
byr invalid: 2003

hgt valid:   60in
hgt valid:   190cm
hgt invalid: 190in
hgt invalid: 190

hcl valid:   #123abc
hcl invalid: #123abz
hcl invalid: 123abc

ecl valid:   brn
ecl invalid: wat

pid valid:   000000001
pid invalid: 0123456789
```

Here are some invalid passports:

```
eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007
```

Here are some valid passports:

```
pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022

iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719
```
Count the number of valid passports - those that have all required fields and valid values. Continue to treat cid as optional. In your batch file, how many passports are valid?

In [5]:
from pydantic import BaseModel, ValidationError, validator
from typing import Optional
from enum import Enum
import re


class EyeColorEnum(str, Enum):
    amb = "amb"
    blu = "blu"
    brn = "brn"
    gry = "gry"
    grn = "grn"
    hzl = "hzl"
    oth = "oth"

class PassportModel(BaseModel):
    byr: str
    iyr: str
    eyr: str
    hgt: str
    hcl: str
    ecl: str
    pid: str
    cid: Optional[str]

class PasswordChecked(PassportModel):

    @validator('byr', always=True)
    def check_byr(cls, val):
        if not (1920 <= int(val) <= 2002):
            raise ValueError
        return val
    
    @validator('iyr', always=True)
    def check_iyr(cls, val):
        if not (2010 <= int(val) <= 2020):
            raise ValueError
        return val
    
    @validator('eyr', always=True)
    def check_eyr(cls, val):
        if not (2020 <= int(val) <= 2030):
            raise ValueError
        return val
    
    @validator('hgt', always=True)
    def check_hgt(cls, val):
        const = {
            "cm": (150, 193),
            "in": (59, 76)
        }
        if val[-2:] not in const.keys():
            raise ValueError
        left, right = const[val[-2:]]
        if not (left <= int(val[:-2]) <= right):
            raise ValueError
        return val
    
    @validator('hcl', always=True)
    def check_hcl(cls, val):
        regex = re.compile(r"#[0-9a-f]{6}")
        if regex.fullmatch(val) is None:
            raise ValueError
        return val
    
    @validator('ecl', always=True)
    def check_ecl(cls, val):
        try:
            EyeColorEnum[val]
        except KeyError:
            raise ValueError
        return val
    
    @validator('pid', always=True)
    def check_pid(cls, val):
        regex = re.compile(r"[0-9]{9}")
        if regex.fullmatch(val) is None:
            raise ValueError
        return val

class Day4Solution(Solution):
    day = 4
    
    def process_input(self, inputs):
        passports = []
        tmp_data = {}
        input_len = len(inputs)
        for index, line in enumerate(inputs):
            line = line.rstrip("\n")
            data = line.split()
            for item in data:
                key, val = item.split(":")
                tmp_data[key] = val
            if line == "" or index == input_len - 1:
                passports.append(tmp_data)
                tmp_data = {}
                continue
        return passports
    
    def count_valid(self, model_cls):
        count = 0
        for input in self.inputs:
            try:
                v = model_cls(**input)
                count += 1
            except ValidationError:
                continue
        return count

    
    def task1(self):
        return self.count_valid(PassportModel)
        
    def task2(self):
        return self.count_valid(PasswordChecked)
Day4Solution.run()

Task 1: 182
Task 2: 109


# Day 5 solutions

## Part1
You board your plane only to discover a new problem: you dropped your boarding pass! You aren't sure which seat is yours, and all of the flight attendants are busy with the flood of people that suddenly made it through passport control.

You write a quick program to use your phone's camera to scan all of the nearby boarding passes (your puzzle input); perhaps you can find your seat through process of elimination.

Instead of zones or groups, this airline uses binary space partitioning to seat people. A seat might be specified like FBFBBFFRLR, where F means "front", B means "back", L means "left", and R means "right".

The first 7 characters will either be F or B; these specify exactly one of the 128 rows on the plane (numbered 0 through 127). Each letter tells you which half of a region the given seat is in. Start with the whole list of rows; the first letter indicates whether the seat is in the front (0 through 63) or the back (64 through 127). The next letter indicates which half of that region the seat is in, and so on until you're left with exactly one row.

For example, consider just the first seven characters of FBFBBFFRLR:

    Start by considering the whole range, rows 0 through 127.
    F means to take the lower half, keeping rows 0 through 63.
    B means to take the upper half, keeping rows 32 through 63.
    F means to take the lower half, keeping rows 32 through 47.
    B means to take the upper half, keeping rows 40 through 47.
    B keeps rows 44 through 47.
    F keeps rows 44 through 45.
    The final F keeps the lower of the two, row 44.

The last three characters will be either L or R; these specify exactly one of the 8 columns of seats on the plane (numbered 0 through 7). The same process as above proceeds again, this time with only three steps. L means to keep the lower half, while R means to keep the upper half.

For example, consider just the last 3 characters of FBFBBFFRLR:

    Start by considering the whole range, columns 0 through 7.
    R means to take the upper half, keeping columns 4 through 7.
    L means to take the lower half, keeping columns 4 through 5.
    The final R keeps the upper of the two, column 5.

So, decoding FBFBBFFRLR reveals that it is the seat at row 44, column 5.

Every seat also has a unique seat ID: multiply the row by 8, then add the column. In this example, the seat has ID 44 * 8 + 5 = 357.

Here are some other boarding passes:

    BFFFBBFRRR: row 70, column 7, seat ID 567.
    FFFBBBFRRR: row 14, column 7, seat ID 119.
    BBFFBBFRLL: row 102, column 4, seat ID 820.

As a sanity check, look through your list of boarding passes. What is the highest seat ID on a boarding pass?

## Part 2

`Ding!` The "fasten seat belt" signs have turned on. Time to find your seat.

It's a completely full flight, so your seat should be the only missing boarding pass in your list. However, there's a catch: some of the seats at the very front and back of the plane don't exist on this aircraft, so they'll be missing from your list as well.

Your seat wasn't at the very front or back, though; the seats with IDs +1 and -1 from yours will be in your list.

What is the ID of your seat?

In [6]:
class Day5Solution(Solution):
    day = 5
    
    def process_input(self, inputs):
        return list(map(lambda x: list(x.rstrip("\n")), inputs))
    
    def binary_distinct(self, left_sign, length, instructions):
        rows = list(range(0, length))
        for instruction in instructions:
            mid = len(rows) // 2
            if instruction == left_sign:
                rows = rows[:mid]
            else:
                rows = rows[mid:]
        return rows[0]

    
    def task1(self):
        hsid = 0
        for input in self.inputs:
            rows_info = input[:7]
            seat_info = input[7:]
            row = self.binary_distinct(
                left_sign="F",
                length=128,
                instructions=rows_info
            )
            seat = self.binary_distinct(
                left_sign="L",
                length=8,
                instructions=seat_info
            )
            seat_uid = (row * 8) + seat
            if seat_uid > hsid:
                hsid = seat_uid
        return hsid
    
    def task2(self):
        plane_matrix = [[0 for _ in range(8)]for _ in range(128)]
        for input in self.inputs:
            rows_info = input[:7]
            seat_info = input[7:]
            row = self.binary_distinct(
                left_sign="F",
                length=128,
                instructions=rows_info
            )
            seat = self.binary_distinct(
                left_sign="L",
                length=8,
                instructions=seat_info
            )
            plane_matrix[row][seat] = 1
        for i, row in enumerate(plane_matrix):
            if all([el == 0 for el in row]):
                continue
            for j, col in enumerate(row):
                if j - 1 < 0:
                    left = plane_matrix[i-1][-1]
                else:
                    left = row[j - 1]
                if j+1 >= len(row):
                    right = plane_matrix[i+1][0]
                else:
                    right = row[j+1]
                if col == 0 and left == 1 and right == 1:
                    return (i * 8) + j
Day5Solution.run()

Task 1: 835
Task 2: 649


# Day 6 solution

## Part 1
As your flight approaches the regional airport where you'll switch to a much larger plane, customs declaration forms are distributed to the passengers.

The form asks a series of 26 yes-or-no questions marked a through z. All you need to do is identify the questions for which anyone in your group answers "yes". Since your group is just you, this doesn't take very long.

However, the person sitting next to you seems to be experiencing a language barrier and asks if you can help. For each of the people in their group, you write down the questions for which they answer "yes", one per line. For example:

abcx
abcy
abcz

In this group, there are 6 questions to which anyone answered "yes": a, b, c, x, y, and z. (Duplicate answers to the same question don't count extra; each question counts at most once.)

Another group asks for your help, then another, and eventually you've collected answers from every group on the plane (your puzzle input). Each group's answers are separated by a blank line, and within each group, each person's answers are on a single line. For example:
```
abc

a
b
c

ab
ac

a
a
a
a

b
```
This list represents answers from five groups:

- The first group contains one person who answered "yes" to 3 questions: a, b, and c.
- The second group contains three people; combined, they answered "yes" to 3 questions: a, b, and c.
- The third group contains two people; combined, they answered "yes" to 3 questions: a, b, and c.
- The fourth group contains four people; combined, they answered "yes" to only 1 question, a.
- The last group contains one person who answered "yes" to only 1 question, b.

In this example, the sum of these counts is 3 + 3 + 3 + 1 + 1 = 11.

For each group, count the number of questions to which anyone answered "yes". What is the sum of those counts?

In [7]:
class Day6Solution(Solution):
    day = 6
    
    def process_input(self, inputs):
        groups = list()
        group = list()
        inputs_len = len(inputs)
        for index, line in enumerate(inputs):
            line = line.rstrip("\n")
            if line == "":
                groups.append(group)
                group = list()
                continue
            group.append(line)
            if index == inputs_len - 1:
                groups.append(group)
                group = list()
        return groups
    
    def task1(self):
        count = 0
        for input in self.inputs:
            count += len(set("".join(input)))
        return count
    
    def task2(self):
        count = 0
        for group in self.inputs:
            first = group[0]
            for answer in first:
                if all([answer in ans_str for ans_str in group]):
                    count += 1
        return count
Day6Solution.run()

Task 1: 6351
Task 2: 3143


# Day 7 solutions

# Part 1

You land at the regional airport in time for your next flight. In fact, it looks like you'll even have time to grab some food: all flights are currently delayed due to issues in luggage processing.

Due to recent aviation regulations, many rules (your puzzle input) are being enforced about bags and their contents; bags must be color-coded and must contain specific quantities of other color-coded bags. Apparently, nobody responsible for these regulations considered how long they would take to enforce!

For example, consider the following rules:

```
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
```

These rules specify the required contents for 9 bag types. In this example, every faded blue bag is empty, every vibrant plum bag contains 11 bags (5 faded blue and 6 dotted black), and so on.

You have a shiny gold bag. If you wanted to carry it in at least one other bag, how many different bag colors would be valid for the outermost bag? (In other words: how many colors can, eventually, contain at least one shiny gold bag?)

In the above rules, the following options would be available to you:

- A bright white bag, which can hold your shiny gold bag directly.
- A muted yellow bag, which can hold your shiny gold bag directly, plus some other bags.
- A dark orange bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.
- A light red bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.

So, in this example, the number of bag colors that can eventually contain at least one shiny gold bag is 4.

How many bag colors can eventually contain at least one shiny gold bag? (The list of rules is quite long; make sure you get all of it.)

## Part 2

It's getting pretty expensive to fly these days - not because of ticket prices, but because of the ridiculous number of bags you need to buy!

Consider again your shiny gold bag and the rules from the above example:

- faded blue bags contain 0 other bags.
- dotted black bags contain 0 other bags.
- vibrant plum bags contain 11 other bags: 5 faded blue bags and 6 dotted black bags.
- dark olive bags contain 7 other bags: 3 faded blue bags and 4 dotted black bags.


So, a single shiny gold bag must contain 1 dark olive bag (and the 7 bags within it) plus 2 vibrant plum bags (and the 11 bags within each of those): 1 + 1*7 + 2 + 2*11 = 32 bags!

Of course, the actual rules have a small chance of going several levels deeper than this example; be sure to count all of the bags, even if the nesting becomes topologically impractical!

Here's another example:

```
shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.
```

In this example, a single shiny gold bag must contain 126 other bags.

How many individual bags are required inside your single shiny gold bag?

In [8]:
import re

from pydantic import BaseModel
from collections import defaultdict, deque

class InnerBag(BaseModel):
    color: str
    quantity: int

class ExternalBag(BaseModel):
    color: str
    inner_bags: List[InnerBag]

    def can_hold(self, color: str) -> bool:
        for bag in self.inner_bags:
            if bag.color == color and bag.quantity > 0:
                return True
        return False
        

class Day7Solution(Solution):
    day = 7

    def process_input(self, inputs: List[str]) -> List[ExternalBag]:
        bags_list = list()
        inner_bag_regex = re.compile(r" ?(?P<quantity>\d+) (?P<color>\w+ \w+) bags?[,.]")
        for input_str in inputs:
            bag_info = input_str.split()
            color = f"{bag_info[0]} {bag_info[1]}"
            inner_bags_info = bag_info[4:]
            inner_bags = []
            for match in inner_bag_regex.finditer(" ".join(inner_bags_info)):
                inner_bags.append(InnerBag(
                    color=match.group("color"),
                    quantity=match.group("quantity")
                ))
            bags_list.append(ExternalBag(
                color=color,
                inner_bags=inner_bags
            ))

        return bags_list

    def task1(self):
        bag_map = defaultdict(list)
        bag_queue = deque()
        counter = set()
        bag_queue.append("shiny gold")
        for input_bag in self.inputs:
            for bag in input_bag.inner_bags:
                bag_map[bag.color].append(input_bag.color)
        
        while bag_queue:
            check_bag = bag_queue.pop()
            parents = bag_map[check_bag]
            counter.update(parents)
            bag_queue.extend(parents)

        return len(counter)

    def task2(self):
        bag_map = defaultdict(list)
        for input_bag in self.inputs:
            for inner_bag in input_bag.inner_bags:
                bag_map[input_bag.color].append(inner_bag)
        
        def traverse(bag: InnerBag):
            children = bag_map[bag.color]
            counter = 1
            for child_bag in children:
                counter += traverse(child_bag)
            return counter * bag.quantity
        
        shiny_bag = InnerBag(color="shiny gold", quantity=1)
        return traverse(shiny_bag) - 1

Day7Solution.run()

Task 1: 213
Task 2: 38426


# Day 8 Solutions

## Part 1

Your flight to the major airline hub reaches cruising altitude without incident. While you consider checking the in-flight menu for one of those drinks that come with a little umbrella, you are interrupted by the kid sitting next to you.

Their handheld game console won't turn on! They ask if you can take a look.

You narrow the problem down to a strange infinite loop in the boot code (your puzzle input) of the device. You should be able to fix it, but first you need to be able to run the code in isolation.

The boot code is represented as a text file with one instruction per line of text. Each instruction consists of an operation (`acc`, `jmp`, or `nop`) and an argument (a signed number like +4 or -20).

- `acc` increases or decreases a single global value called the accumulator by the value given in the argument. For example, `acc +7` would increase the accumulator by 7. The accumulator starts at 0. After an acc instruction, the instruction immediately below it is executed next.
- `jmp` jumps to a new instruction relative to itself. The next instruction to execute is found using the argument as an offset from the `jmp` instruction; for example, `jmp +2` would skip the next instruction, `jmp +1` would continue to the instruction immediately below it, and `jmp -20` would cause the instruction 20 lines above to be executed next.
- `nop` stands for No OPeration - it does nothing. The instruction immediately below it is executed next.

For example, consider the following program:
```
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
```
These instructions are visited in this order:
```
nop +0  | 1
acc +1  | 2, 8(!)
jmp +4  | 3
acc +3  | 6
jmp -3  | 7
acc -99 |
acc +1  | 4
jmp -4  | 5
acc +6  |
```
First, the `nop +0` does nothing. Then, the accumulator is increased from 0 to 1 (`acc +1`) and `jmp +4` sets the next instruction to the other `acc +1` near the bottom. After it increases the accumulator from 1 to 2, `jmp -4` executes, setting the next instruction to the only `acc +3`. It sets the accumulator to 5, and `jmp -3` causes the program to continue back at the first `acc +1`.

This is an infinite loop: with this sequence of jumps, the program will run forever. The moment the program tries to run any instruction a second time, you know it will never terminate.

Immediately before the program would run an instruction a second time, the value in the accumulator is 5.

Run your copy of the boot code. Immediately before any instruction is executed a second time, what value is in the accumulator?

## Part 2

After some careful analysis, you believe that exactly one instruction is corrupted.

Somewhere in the program, either a `jmp` is supposed to be a `nop`, or a `nop` is supposed to be a `jmp`. (No `acc` instructions were harmed in the corruption of this boot code.)

The program is supposed to terminate by attempting to execute an instruction immediately after the last instruction in the file. By changing exactly one `jmp` or `nop`, you can repair the boot code and make it terminate correctly.

For example, consider the same program from above:
```
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
```

If you change the first instruction from `nop +0` to `jmp +0`, it would create a single-instruction infinite loop, never leaving that instruction. If you change almost any of the jmp instructions, the program will still eventually find another jmp instruction and loop forever.

However, if you change the second-to-last instruction (from `jmp -4` to `nop -4`), the program terminates! The instructions are visited in this order:
```
nop +0  | 1
acc +1  | 2
jmp +4  | 3
acc +3  |
jmp -3  |
acc -99 |
acc +1  | 4
nop -4  | 5
acc +6  | 6
```
After the last instruction (`acc +6`), the program terminates by attempting to run the instruction below the last instruction in the file. With this change, after the program terminates, the accumulator contains the value 8 (`acc +1`, `acc +1`, `acc +6`).

Fix the program so that it terminates normally by changing exactly one `jmp` (to `nop`) or `nop` (to `jmp`). What is the value of the accumulator after the program terminates?

In [9]:
from typing import List
from pydantic import BaseModel
from pprint import pprint
from enum import Enum
from copy import copy

class Command(str, Enum):
    acc = "acc"
    jmp = "jmp"
    nop = "nop"

class Instruciton(BaseModel):
    command: Command
    value: int

class Day8Solution(Solution):
    day = 8

    def process_input(self, inputs) -> List[Instruciton]:
        instructions = []
        for line in inputs:
            command, value = line.split()
            instructions.append(Instruciton(
                command=command,
                value=value
            ))
        return instructions

    def do_step(self, index: int, acc: int, instruction: Instruciton):
            if instruction.command == Command.acc:
                acc += instruction.value
                index += 1
            elif instruction.command == Command.jmp:
                index += instruction.value
            else:
                index += 1
            return index, acc
  
    def check_loop(self, instructions_list: List[Instruciton]):
        acc = 0
        current_index = 0
        visited_indices = list()
        has_cycle = False
        while True:
            if current_index in visited_indices:
                has_cycle = True
                break
            if current_index >= len(instructions_list):
                has_cycle = False
                break
            instruction = instructions_list[current_index]
            visited_indices.append(current_index)
            previous_index = current_index
            current_index, acc = self.do_step(
                current_index,
                acc,
                instruction
            )
        return has_cycle, acc, visited_indices

    def task1(self):
        _, acc, _ = self.check_loop(self.inputs)
        return acc


    def task2(self):        
        for instruction_index, instruction in enumerate(self.inputs):
            instructions_copy = copy(self.inputs)
            was_changed = False
            if instruction.command in (Command.jmp, Command.nop):
                new_command = Command.nop
                if instruction.command == Command.nop:
                    new_command = Command.jmp
                instructions_copy[instruction_index] = Instruciton(
                    command=Command.nop,
                    value=instruction.value
                )
                has_cycle, acc, trace = self.check_loop(instructions_copy)
                if not has_cycle:
                    return acc
        print("Task can't be solved.")
        return None


Day8Solution.run()

Task 1: 1317
Task 2: 1033


# Day 9 solutions

## Task 1

With your neighbor happily enjoying their video game, you turn your attention to an open data port on the little screen in the seat in front of you.

Though the port is non-standard, you manage to connect it to your computer through the clever use of several paperclips. Upon connection, the port outputs a series of numbers (your puzzle input).

The data appears to be encrypted with the eXchange-Masking Addition System (XMAS) which, conveniently for you, is an old cypher with an important weakness.

XMAS starts by transmitting a preamble of 25 numbers. After that, each number you receive should be the sum of any two of the 25 immediately previous numbers. The two numbers will have different values, and there might be more than one such pair.

For example, suppose your preamble consists of the numbers 1 through 25 in a random order. To be valid, the next number must be the sum of two of those numbers:

    26 would be a valid next number, as it could be 1 plus 25 (or many other pairs, like 2 and 24).
    49 would be a valid next number, as it is the sum of 24 and 25.
    100 would not be valid; no two of the previous 25 numbers sum to 100.
    50 would also not be valid; although 25 appears in the previous 25 numbers, the two numbers in the pair must be different.

Suppose the 26th number is 45, and the first number (no longer an option, as it is more than 25 numbers ago) was 20. Now, for the next number to be valid, there needs to be some pair of numbers among 1-19, 21-25, or 45 that add up to it:

    26 would still be a valid next number, as 1 and 25 are still within the previous 25 numbers.
    65 would not be valid, as no two of the available numbers sum to it.
    64 and 66 would both be valid, as they are the result of 19+45 and 21+45 respectively.

Here is a larger example which only considers the previous 5 numbers (and has a preamble of length 5):

```
35
20
15
25
47
40
62
55
65
95
102
117
150
182
127
219
299
277
309
576
```

In this example, after the 5-number preamble, almost every number is the sum of two of the previous 5 numbers; the only number that does not follow this rule is 127.

The first step of attacking the weakness in the XMAS data is to find the first number in the list (after the preamble) which is not the sum of two of the 25 numbers before it. What is the first number that does not have this property?

## Task 2

The final step in breaking the XMAS encryption relies on the invalid number you just found: you must find a contiguous set of at least two numbers in your list which sum to the invalid number from step 1.

Again consider the above example:

```
35
20
15
25
47
40
62
55
65
95
102
117
150
182
127
219
299
277
309
576
```

In this list, adding up all of the numbers from 15 through 40 produces the invalid number from step 1, 127. (Of course, the contiguous set of numbers in your actual list might be much longer.)

To find the encryption weakness, add together the smallest and largest number in this contiguous range; in this example, these are 15 and 47, producing 62.

What is the encryption weakness in your XMAS-encrypted list of numbers?

In [18]:
class Day9Solution(Solution):
    day = 9

    def process_input(self, inputs):
        return list(map(int, inputs))

    def task1(self):
        index = 25
        window = self.inputs[:index]
        while index < len(self.inputs):
            target = self.inputs[index]
            try:
                for i in window:
                    for j in window:
                        if i + j == target:
                            found = True
                            index += 1
                            window.pop(0)
                            window.append(target)
                            raise ValueError("Found")
            except ValueError:
                continue
            return target

    def task2(self):
        weak_number = self.task1()
        range_start = 0
        range_end = 1
        found = False
        while True:
            range_sum = sum(map(lambda x: self.inputs[x], range(range_start, range_end)))
            if range_sum == weak_number:
                break
            elif range_sum < weak_number:
                range_end += 1
            elif range_sum > weak_number:
                range_start += 1
        target_sequence = sorted(list(map(lambda x: self.inputs[x], range(range_start, range_end))))
        return target_sequence[0] + target_sequence[-1]
Day9Solution.run()

Task 1: 373803594
Task 2: 51152360
